Функция для чтения файла. Принимает путь к файлу, возвращает содержимое файла как строку

In [2]:
def read_file(file_path):
    with open(file_path, encoding = "utf8") as input_file:
        return input_file.read()

Функция для чтения файла со словами для замены. Принимает ссылку на .csv файл, возвращает словарь 

In [3]:
import pandas as pd
import requests
import io
    
def read_change_words_dict(file_url):
    download = requests.get(file_url).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    change_words_dict = dict(zip(df['from'], df['to']))

    # добавляю пару ёж-страус, потому что в статье про ежа ё с точками используется
    change_words_dict['ёж'] = 'страус'

    # убираю закравшийся в "субкультуры " пробел
    change_words_dict['субкультуры'] = change_words_dict.pop('субкультуры ')

    return change_words_dict

Функция для замены слов в тексте. Принимает текст и словарь со словами для замены, возвращает отредактированный текст как строку

In [4]:
import re

def change_words_in_text(text, change_words_dict):
    
    edited_text = text
    
    for word in change_words_dict.keys():    
        # выбираем слова, для которых символ справа-слева - это не буквы    
        change_list = re.findall(rf'[^а-яА-я]{word}[^а-яА-я]', text, re.IGNORECASE)
        change_set = set(change_list)

        for pattern in change_set:
            # не обрабатываем однобуквенные предлоги\союзы с точкой,
            # которые (скорее всего) инициал (В.Иванова, там, или И.Васильева)
            if(len(word) == 1 and re.search(r'\.$', pattern) != None): 
                continue

            # добавляем обработку для заглавных букв и составляем замену с учетом символов справа-слева
            if pattern.islower():
                replacement = re.sub(word, change_words_dict[word], pattern)
            else:
                change_word = change_words_dict[word][0].upper() + change_words_dict[word][1:]
                replacement = re.sub(word[0].upper() + word[1:], change_word, pattern, re.IGNORECASE)

            # для строк, где встречаются спецальные символы, делаем их обычными символами
            pattern = re.sub(r'\)', '\)', pattern)
            pattern = re.sub(r'\(', '\(', pattern)
            pattern = re.sub(r'\]', '\]', pattern)
            pattern = re.sub(r'\[', '\[', pattern)
            pattern = re.sub(r'\.', '\.', pattern)
            
            edited_text = re.sub(pattern, replacement, edited_text)            
    return edited_text

Функция для сохранения файла. Принимает путь к файлу и текст, который надо записать

In [5]:
def create_file(file_path, text):
    with open(file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(text)

Основая часть - изменить пути к файлам, если надо

In [8]:
original_files = ['LS_Py_Tasks/wiki/hedgehog_original.txt', 'LS_Py_Tasks/wiki/goth_original.txt', 'LS_Py_Tasks/wiki/tongue_original.txt']
edited_files = ['LS_Py_Tasks/wiki/hedgehog_edited.txt', 'LS_Py_Tasks/wiki/goth_edited.txt', 'LS_Py_Tasks/wiki/tongue_edited.txt']

# RAW URL!!!
change_words_csv_url = "https://raw.githubusercontent.com/sergeychuvakin/ML-TEXTS/master/intro/changes.csv"

for i in range (0, 3):
    text = read_file(original_files[i])
    edited_text = change_words_in_text(text, read_change_words_dict(change_words_csv_url))
    create_file(edited_files[i], edited_text)